In [1]:
import json
import os
import itertools
from typing import Tuple

import pandas as pd
import numpy as np
import talib as ta
from tqdm import tqdm

from ftx_client import FtxClient
import supertrend as spt
import backtesting as bt
from config import API_KEY, API_SECRET

In [2]:
with open("settings.json") as jsonfile:
    settings = json.load(jsonfile)
    
BACKTEST_FOLDER = settings["filepaths"]["backtest_folder"]
OPTIMIZEDML_FILEPATH = os.path.join(BACKTEST_FOLDER, settings["filepaths"]["optimized_ml_file"])
ANALYSIS_FILEPATH = os.path.join(BACKTEST_FOLDER, settings["filepaths"]["analysis_file"])

In [3]:
ftx = FtxClient(api_key=API_KEY, api_secret=API_SECRET)

In [4]:
markets = []
for future in ftx.list_futures():
    if future["type"] == "perpetual":
        if (future["volumeUsd24h"] > settings["markets"]["min_volume_usd_24h"] and 
                future["name"] not in settings["markets"]["blacklist"]):
            markets.append(future["name"])

In [5]:
optimzed_ml = pd.read_csv(OPTIMIZEDML_FILEPATH)

In [6]:
analysis_df = pd.DataFrame(columns=["Name", "AvgReturns", "StdDev", "AvgDrawdown", "RetDevRatio", "MinReturns", "MaxReturns",
             "AvgNegReturns", "AvgPosReturns", "PosNegRetRatio", "MedReturns", "MedNegReturns",
             "MedPosReturns", "MedPosNegRetRatio", "TheDfactor"])

pbar = tqdm(markets)
for market in pbar:
    pbar.set_description(market)
    df = ftx.get_historical_market_data(market, interval=settings["analysis"]["interval"], start_time=settings["analysis"]["start_time"])
    
    if len(df) < settings["analysis"]["min_data_length"]:
        continue
        
    multiplier = optimzed_ml.loc[optimzed_ml['Name'] == market]["Multiplier"].values[0]
    lookback = optimzed_ml.loc[optimzed_ml['Name'] == market]["Lookback"].values[0]
    
    profits = bt.backtest_dataframe(df, look_back=lookback, multiplier=multiplier)
    profits["Name"] = market
    analysis_df = analysis_df.append(profits, ignore_index=True)

XRP-PERP: 100%|██████████| 35/35 [01:48<00:00,  3.10s/it]  


In [7]:
analysis_df = analysis_df.round(1)
analysis_df.to_csv(ANALYSIS_FILEPATH, index=False)

In [8]:
pd.read_csv(ANALYSIS_FILEPATH).sort_values("TheDfactor", ascending=False) 

,Name,AvgReturns,StdDev,AvgDrawdown,RetDevRatio,MinReturns,MaxReturns,AvgNegReturns,AvgPosReturns,PosNegRetRatio,MedReturns,MedNegReturns,MedPosReturns,MedPosNegRetRatio,TheDfactor
2,ALPHA-PERP,89.8,128.8,4.9,0.7,0.7,344.8,0.0,89.8,89.8,27.1,0.0,27.1,27.1,1.1
23,LUNA-PERP,73.4,92.2,4.7,0.8,1.0,203.5,0.0,73.4,73.4,15.5,0.0,15.5,15.5,1.1
33,XRP-PERP,9.8,22.6,9.3,0.4,-13.3,58.1,-4.3,38.0,8.9,-1.0,-2.3,39.9,17.3,0.2
7,BCH-PERP,23.3,32.8,3.7,0.7,-5.4,108.7,-3.6,30.9,8.7,12.9,-3.6,22.8,6.4,0.2
26,SAND-PERP,47.9,49.1,15.0,1.0,-11.2,104.9,-7.1,84.6,11.9,49.1,-7.1,100.0,14.0,0.2
8,BNB-PERP,25.2,35.6,7.3,0.7,1.2,95.6,0.0,25.2,25.2,8.6,0.0,8.6,8.6,0.2
31,SXP-PERP,41.1,75.3,4.3,0.5,-5.8,190.8,-3.7,71.0,19.2,2.9,-3.7,19.2,5.2,0.2
27,SNX-PERP,27.1,33.5,10.0,0.8,-10.3,90.7,-6.0,40.4,6.7,23.0,-6.0,30.9,5.1,0.1
29,SRM-PERP,13.7,24.2,6.2,0.6,-15.9,71.8,-7.3,31.7,4.3,6.4,-6.3,25.5,4.1,0.1
18,GRT-PERP,12.8,25.2,6.7,0.5,-11.1,90.4,-4.8,23.8,4.9,4.1,-3.3,19.0,5.8,0.1
